<a href="https://colab.research.google.com/github/sifatbhuiyan0909/Dhaka-Finance-Navigator/blob/main/notebooks/10_final_modeling_and_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded= files.upload()

Saving Dhaka-Stock-Exchange-DSE-2021.csv to Dhaka-Stock-Exchange-DSE-2021.csv
Saving Dhaka-Stock-Exchange-DSE-2020.csv to Dhaka-Stock-Exchange-DSE-2020.csv
Saving Dhaka-Stock-Exchange-DSE-2019.csv to Dhaka-Stock-Exchange-DSE-2019.csv
Saving Dhaka-Stock-Exchange-DSE-2018.csv to Dhaka-Stock-Exchange-DSE-2018.csv
Saving Dhaka-Stock-Exchange-DSE-2017.csv to Dhaka-Stock-Exchange-DSE-2017.csv
Saving Dhaka-Stock-Exchange-DSE-2016.csv to Dhaka-Stock-Exchange-DSE-2016.csv
Saving Dhaka-Stock-Exchange-DSE-2015.csv to Dhaka-Stock-Exchange-DSE-2015.csv
Saving Dhaka-Stock-Exchange-DSE-2014.csv to Dhaka-Stock-Exchange-DSE-2014.csv
Saving Dhaka-Stock-Exchange-DSE-2013.csv to Dhaka-Stock-Exchange-DSE-2013.csv
Saving Dhaka-Stock-Exchange-DSE-2012.csv to Dhaka-Stock-Exchange-DSE-2012.csv
Saving Dhaka-Stock-Exchange-DSE-2011.csv to Dhaka-Stock-Exchange-DSE-2011.csv
Saving Dhaka-Stock-Exchange-DSE-2010.csv to Dhaka-Stock-Exchange-DSE-2010.csv
Saving Dhaka-Stock-Exchange-DSE-2009.csv to Dhaka-Stock-Exchange

In [2]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing inst

In [2]:
import pandas as pd
import glob
import os
import numpy as np
import pandas_ta as ta
from sklearn.model_selection import train_test_split

# --- Day 3-8 Setup ---
print("--- Initializing Data Pipeline (Days 3-8) ---")
all_filenames = glob.glob(os.path.join('.', '*.csv'))
master_df = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], ignore_index=True)
master_df.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df = master_df.copy()

# Day 4: Cleaning
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
for col in ['Open', 'Low', 'Volume']:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace('-', '', regex=False), errors='coerce')
df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')
df = df.set_index('Date')

# Day 5 & 6: Imputation and Outlier Correction
cols_to_fill = ['Open', 'High', 'Low', 'Close', 'Volume']
df[cols_to_fill] = df[cols_to_fill].ffill()
df['Daily_Return'] = df['Close'].pct_change()
mu, sigma = df['Daily_Return'].mean(), df['Daily_Return'].std()
outlier_mask = (df['Daily_Return'] < mu - 3 * sigma) | (df['Daily_Return'] > mu + 3 * sigma)
df.loc[outlier_mask, cols_to_fill] = np.nan
df[cols_to_fill] = df[cols_to_fill].ffill()
df = df.drop(columns=['Daily_Return'])

# Day 7: Base Feature Engineering
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# Day 8: Advanced Feature Engineering (Groupby Fix)
df = df.reset_index()
def create_advanced_features(group):
    group['RSI'] = ta.rsi(close=group['Close'], length=14)
    macd_result = ta.macd(close=group['Close'])
    if macd_result is not None:
        if 'MACD_12_26_9' in macd_result.columns:
            group['MACD'] = macd_result['MACD_12_26_9']
        else:
            group['MACD'] = macd_result.iloc[:, 0]
    else:
        group['MACD'] = np.nan
    group['Lag_Log_Return'] = group['Log_Return'].shift(1)
    return group.dropna()
df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features)
df = df.set_index('Date')
print("--- Data Preparation (Days 3-8) Complete ---")

# ==============================================================================
# 📅 Day 9: Target Creation and Data Splitting
# ==============================================================================
print("\n--- Day 9: Target Creation and Data Splitting ---")
def create_target(group):
    future_price = group['Close'].shift(-1)
    target = (future_price > group['Close']).astype(int)
    return group.assign(Target=target).dropna(subset=['Target'])

df = df.groupby('Ticker', group_keys=False).apply(create_target)

# Define X and y
X = df.drop(columns=['Target', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker'])
y = df['Target']

# Chronological Split (shuffle=False is essential)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False, stratify=None
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print("--- Day 9 Complete. Ready for Modeling. ---")

--- Initializing Data Pipeline (Days 3-8) ---


/tmp/ipython-input-2657379518.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features)
/tmp/ipython-input-2657379518.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features)


--- Data Preparation (Days 3-8) Complete ---

--- Day 9: Target Creation and Data Splitting ---
X_train shape: (948163, 6)
X_test shape: (237041, 6)
--- Day 9 Complete. Ready for Modeling. ---


/tmp/ipython-input-2657379518.py:64: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Ticker', group_keys=False).apply(create_target)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# NOTE: X_train, X_test, y_train, y_test must be loaded in memory from the previous step (Day 9).

print("\n--- Day 10: Initial Model Training and Evaluation (Baseline) ---")

# --- 1. Initialize and Train the Model ---
# We use the RandomForestClassifier as a strong, reliable benchmark model.
# n_estimators=100 means 100 decision trees will be built (the "forest").
# n_jobs=-1 tells the model to use all available processor cores for fast training.
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

print("Training Random Forest Classifier on historical data...")
# The .fit() function is the learning step: the model finds patterns in X_train to predict y_train.
model.fit(X_train, y_train)

# --- 2. Predict on the Test Set ---
# The model makes predictions (y_pred) on the unseen future data (X_test).
y_pred = model.predict(X_test)

# --- 3. Evaluation ---
accuracy = accuracy_score(y_test, y_pred)
# classification_report provides detailed metrics like Precision and Recall.
report = classification_report(y_test, y_pred, zero_division=0)

print("\n--- Evaluation Results ---")
print(f"Overall Accuracy on Test Set: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(report)

print("\nDay 10 Complete. Baseline Model trained and evaluated.")


--- Day 10: Initial Model Training and Evaluation (Baseline) ---
Training Random Forest Classifier on historical data...

--- Evaluation Results ---
Overall Accuracy on Test Set: 57.68%

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.79      0.69    139810
           1       0.47      0.28      0.35     97231

    accuracy                           0.58    237041
   macro avg       0.54      0.53      0.52    237041
weighted avg       0.55      0.58      0.55    237041


Day 10 Complete. Baseline Model trained and evaluated.


its a shame that the accuracy is 57.68% but we have to think about the factors in the share market like war, human emotion and so on.